In [1]:
from ultralytics import YOLO
import torch
from torchvision import transforms
from PIL import Image


def crop_left(im): 
    return transforms.functional.crop(im, top=0, left=0, height=420, width=560)


class FeatureExtractor:
    def __init__(self, model):
        self.model = model
        self.features = []

    def hook(self, module, input, output):
        self.features.append(output)

    def register_hooks(self, layer_names):
        self.hooks = []
        for name, module in self.model.named_modules():
            if name in layer_names:
                hook = module.register_forward_hook(self.hook)
                self.hooks.append(hook)

    def remove_hooks(self):
        for hook in self.hooks:
            hook.remove()

    def extract_features(self, img_path):
        self.features = []
        img = Image.open(img_path).convert('RGB')  # Load image using PIL
        transform = transforms.Compose([
            transforms.Lambda(crop_left),  # Crop the left side
            transforms.Pad(padding=(40, 110, 40, 110)), # Pad to 640x640
            transforms.ToTensor()
        ])
        img = transform(img).unsqueeze(0)  # Transform to tensor and add batch dimension
        self.model(img)
        return self.features

# Initialize YOLO model
model = YOLO('yolov8m.pt')

# Initialize the feature extractor
feature_extractor = FeatureExtractor(model)

# Register hooks on the desired layers (use appropriate layer names)
layer_names = ['model.model.17']  # Example layer names, update accordingly
feature_extractor.register_hooks(layer_names)

In [2]:
# num_demos = 24
# for i in range(num_demos):
#     img_path = f'data/{i}/img.jpeg'
#     features = feature_extractor.extract_features(img_path)
#     # print(f"Features for image {i}: {features}")
#     print(features[0].shape)

# # Remove hooks after extraction
# feature_extractor.remove_hooks()


In [3]:
import os
import csv


t_steps = 400

num_demos = 24

trajs = torch.zeros(num_demos, t_steps, 8)
feats = torch.zeros(num_demos, 576*40*40)

for i in range(num_demos):
    img_path = f'data/{i}/img.jpeg'
    features = feature_extractor.extract_features(img_path)
    feats[i] = features[0].view(-1)

    data_folder = f'/home/yigit/projects/cnep/baxter/data/{i}/'
    # iterate over all files in the in_folder
    for filename in os.listdir(data_folder):
        d = os.path.join(data_folder, filename)
        if filename.endswith('.csv'):
            temp_data = []
            with open(d, 'r') as f:
                for j, line in enumerate(csv.reader(f)):
                    if j > 0:
                        temp_data.append([float(line[3]), float(line[4]), float(line[5]), float(line[6]), float(line[7]), float(line[8]), float(line[9]), float(line[10])])  # p, q, gripper

    ids = torch.linspace(0, len(temp_data)-1, t_steps).int()

    for j in range(t_steps):
        trajs[i, j] = torch.tensor(temp_data[ids[j]])

torch.save(trajs, 'trajs.pt')
torch.save(feats, 'feats.pt')
                        


0: 640x640 1 cup, 12.4ms
Speed: 0.6ms preprocess, 12.4ms inference, 331.2ms postprocess per image at shape (1, 3, 640, 640)


IndexError: list index out of range

In [7]:
ids[j]

tensor(1158, dtype=torch.int32)